# Basic 3WNN Clustering w/ Gated Labels

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
quiet_library(ArchR)
quiet_library(cowplot)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

# Load mnp Subset

In [ ]:
mnp <- readRDS('../Figure5/mnp_analysis_3wnn_updated.rds')

In [ ]:
table(mnp$cell.types)

In [ ]:
mnp <- SetIdent(mnp, value = 'cell.types')
mnp <- subset(mnp, idents = 'MNP-2')

# Add Metadata

In [ ]:
mnp <- SetIdent(mnp, value = 'pbmc_sample_id')
mnp <- RenameIdents(mnp,
                           'PB00173-02' = 'Positive', #validated
                           'PB00192-02' = 'Negative', #validated
                           'PB00197-02' = 'Negative', #validated
                           'PB00593-04' = 'Positive', #validated
                           'PB00172-02' = 'Positive',
                           'PB00182-02' = 'Negative',
                           'PB00323-02' = 'Positive',
                           'PB00807-02' = 'Positive',
                           'PB00124-02' = 'Positive',
                           'PB00127-02' = 'Negative',
                           'PB00334-03' = 'Negative',
                           'PB00353-03' = 'Negative',
                           'PB00368-04' = 'Positive',
                           'PB00377-03' = 'Negative',
                           'PB00545-02' = 'Negative',
                           'PB00599-02' = 'Positive')
mnp$CMV <- Idents(mnp)

In [ ]:
mnp <- SetIdent(mnp, value = 'pbmc_sample_id')
mnp <- RenameIdents(mnp,
                           'PB00173-02' = 'UP1006', #validated
                           'PB00192-02' = 'UP1007', #validated
                           'PB00197-02' = 'UP1010', #validated
                           'PB00593-04' = 'BR2002', #validated
                           'PB00172-02' = 'UP1002',
                           'PB00182-02' = 'UP1003',
                           'PB00323-02' = 'BR2004',
                           'PB00807-02' = 'UP1005',
                           'PB00124-02' = 'UP1001',
                           'PB00127-02' = 'UP1004',
                           'PB00334-03' = 'BR2009',
                           'PB00353-03' = 'BR2008',
                           'PB00368-04' = 'BR2005',
                           'PB00377-03' = 'BR2015',
                           'PB00545-02' = 'BR2042',
                           'PB00599-02' = 'BR2052')
mnp$subject_id <- Idents(mnp)

In [ ]:
mnp$age_cmv <- paste0(mnp$pediatric_senior,"_CMV-",mnp$CMV)

# RNA Clustering

In [ ]:
outlier_genes <- c('AC105402.3','MTRNR2L8','EDA','IFNG-AS1')

In [ ]:
mnp <- suppressWarnings(SCTransform(mnp)) %>% RunPCA()

In [ ]:
var_feat_sct <- mnp@assays$SCT@var.features
length(var_feat_sct)

In [ ]:
outlier_genes %in% mnp@assays$SCT@var.features

In [ ]:
var_feats <- mnp@assays$SCT@var.features
var_feats_trim <- var_feats[!(var_feats %in% outlier_genes)]
length(var_feats_trim)

In [ ]:
mnp <- suppressWarnings(SCTransform(mnp, residual.features = var_feats_trim)) %>% RunPCA()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
mnp <- suppressWarnings(RunHarmony(object = mnp, reduction.save = 'rna_harmony',group.by.vars = 'batch_id', reduction = 'pca', 
                                           plot_convergence = T, assay.use = 'SCT'))

In [ ]:
ElbowPlot(mnp, ndims = 50)

In [ ]:
mnp <- RunUMAP(mnp, dims = 1:20, reduction = 'rna_harmony')

## RNA UMAPs

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(mnp, reduction = 'umap', group.by = 'predicted.t_celltype.l2', shuffle = TRUE)
p2 <- DimPlot(mnp, reduction = 'umap', group.by = 'pediatric_senior', shuffle = TRUE)
p3 <- DimPlot(mnp, reduction = 'umap', group.by = 'age_cmv', shuffle = TRUE)
p4 <- DimPlot(mnp, reduction = 'umap', group.by = 'pbmc_sample_id', shuffle = TRUE)
p1 + p2 + p3 + p4

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)
suppressMessages(FeaturePlot(mnp, features = c('sct_IFNG-AS1','sct_EDA','sct_SOX4','sct_ZNF683'), 
                             ncol = 2,reduction = 'umap', order = TRUE) & scale_color_viridis())

# ADT Clustering

In [ ]:
adts_to_remove <- c('CD134','CD172a','CD183','CD366','CX3CR1','TCRab','TCRgd','CD137','CD24','CD294','CD304','CD40','CD80','CD86','IgG1-K-Isotype-Control')
adt_mtx <- mnp@assays$ADT@counts
length(rownames(adt_mtx))

In [ ]:
idx <- which(rownames(adt_mtx) %in% adts_to_remove)
clean_adt_mtx <- adt_mtx[-idx,]
length(rownames(clean_adt_mtx))

In [ ]:
mnp[['clean_adt']] <- CreateAssayObject(clean_adt_mtx)
DefaultAssay(mnp) <- 'clean_adt'

In [ ]:
# cluster by ADTs, make sure to save under all new reductions and keys 
VariableFeatures(mnp) <- rownames(mnp[["clean_adt"]])
mnp <- NormalizeData(mnp, normalization.method = 'CLR', margin = 2) %>% 
     ScaleData() %>% RunPCA(reduction.name = 'apca', approx = FALSE, reduction.key = 'APC_')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
mnp <- suppressWarnings(RunHarmony(object = mnp, reduction.save = 'adt_harmony',group.by.vars = 'batch_id', reduction = 'apca', 
                                           plot_convergence = T, assay.use = 'clean_adt'))

In [ ]:
mnp <- RunUMAP(mnp, reduction = 'adt_harmony', reduction.name = 'adtumap', dims = 1:20)

## ADT UMAPs

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(mnp, reduction = 'adtumap', group.by = 'predicted.t_celltype.l2')
p2 <- DimPlot(mnp, reduction = 'adtumap', group.by = 'pediatric_senior')
p3 <- DimPlot(mnp, reduction = 'adtumap', group.by = 'age_cmv')
p4 <- DimPlot(mnp, reduction = 'adtumap', group.by = 'pbmc_sample_id')
p1 + p2 + p3 + p4

# ATAC Clustering

## Load ArchR Project

In [ ]:
proj <- loadArchRProject(path = '/home/jupyter/Organized_Gating_Analysis/08_Rework/Figure5/')
proj

## Subset ArchR Project

In [ ]:
bcs <- colnames(mnp[['RNA']])
head(bcs)

In [ ]:
idx_new <- which(proj$cellNames_clean %in% bcs)
head(idx_new)

In [ ]:
cells_subset <- proj$cellNames[idx_new]
proj_subset <- proj[cells_subset,]
proj_subset

## ATAC Clustering

In [ ]:
proj_subset <- addIterativeLSI(proj_subset, name = 'IterativeLSI', force = TRUE, varFeatures = 75000)

In [ ]:
proj_subset <- addHarmony(proj_subset, reducedDims = 'IterativeLSI', groupBy = 'batch_id', force = TRUE)

In [ ]:
#proj_subset <- addClusters(proj_subset, reducedDims = 'IterativeLSI', name = 'Clusters', force = TRUE, resolution = 0.2)
proj_subset <- addClusters(proj_subset, reducedDims = 'Harmony', name = 'Clusters', force = TRUE, resolution = 0.2)

In [ ]:
#proj_subset <- addUMAP(proj_subset, reducedDims = 'IterativeLSI', name = 'UMAP', force = TRUE)
proj_subset <- addUMAP(proj_subset, reducedDims = 'Harmony', name = 'UMAP', force = TRUE)

In [ ]:
options(repr.plomnpidth = 8, repr.plot.height = 8)
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "Clusters", embedding = "UMAP")
# plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "pediatric_senior", embedding = "UMAP")
# plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "predicted.t_celltype.l2", embedding = "UMAP")

## Export LSI

In [ ]:
#lsi <- getReducedDims(proj_subset, reducedDims = 'IterativeLSI')
lsi <- getReducedDims(proj_subset, reducedDims = 'Harmony')

## Import LSI to Seurat Object

In [ ]:
rna_metadata <- mnp@meta.data
archr_style_barcodes <- paste(rna_metadata$batch_id, "-P1_",rna_metadata$pbmc_sample_id,"#",rownames(rna_metadata), sep = "")
head(archr_style_barcodes)

In [ ]:
# create data frame w/ seurat barcodes and archr style barcodes
bc_df <- data.frame(seurat_bcs = colnames(mnp@assays$RNA),
                    archr_style = archr_style_barcodes)

In [ ]:
# Find Intersection & subset LSI
bc_idx <- intersect(rownames(lsi), archr_style_barcodes)

In [ ]:
lsi_subset <- subset(lsi, subset = rownames(lsi) %in% bc_idx)
bc_df_subset <- subset(bc_df, subset = bc_df$archr_style %in% bc_idx)
length(rownames(lsi_subset))
length(rownames(bc_df_subset))

In [ ]:
table(bc_df_subset$archr_style == rownames(lsi_subset))

In [ ]:
lsi_subset <- lsi_subset[match(bc_df_subset$archr_style, rownames(lsi_subset)),]

In [ ]:
table(bc_df_subset$archr_style == rownames(lsi_subset))

In [ ]:
# replace LSI archr barcodes for seurat barcodes
rownames(lsi_subset) <- bc_df_subset$seurat_bcs

In [ ]:
mnp <- subset(mnp, cells = bc_df_subset$seurat_bcs)

In [ ]:
table(colnames(mnp[['RNA']]) == rownames(lsi_subset))

In [ ]:
mnp[["lsit"]] <- CreateDimReducObject(embeddings = lsi_subset, key = "lsit_", assay = "Tiles")

## ATAC clustering in Seurat Object

In [ ]:
DefaultAssay(mnp) <- 'Tiles'
mnp <- RunUMAP(mnp, reduction = 'lsit', reduction.name = 'atac_umap', dims = 1:15, verbose = T, reduction.key = 'atacumap_')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(mnp, reduction = 'atac_umap', group.by = 'predicted.t_celltype.l2')
p2 <- DimPlot(mnp, reduction = 'atac_umap', group.by = 'pediatric_senior')
p3 <- DimPlot(mnp, reduction = 'atac_umap', group.by = 'age_cmv')
p4 <- DimPlot(mnp, reduction = 'atac_umap', group.by = 'pbmc_sample_id')
p1 + p2 + p3 + p4

In [ ]:
#options(repr.plot.width = 20, repr.plot.height = 20)
#DimPlot(mnp, reduction = 'atac_umap', group.by = 'gating_celltype', split.by = 'gating_celltype', ncol = 4)

# 3WNN Clustering

In [ ]:
mnp <- FindMultiModalNeighbors(mnp, reduction.list = list("rna_harmony","adt_harmony",'lsit'), dims.list = list(1:20,1:20,1:29), weighted.nn.name = "X3way.weighted.nn",
                                      k.nn = 30, knn.range = 500, prune.SNN = 1/20)

In [ ]:
mnp <- RunUMAP(mnp, nn.name = "X3way.weighted.nn", reduction.name = "wnn.3.umap", reduction.key = "wnn.3.umap_")

In [ ]:
mnp <- FindClusters(mnp, graph.name = "wsnn", algorithm = 3, resolution = 0.5, verbose = TRUE)

## 3WNN UMAPs

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 8)
p1 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.5', label = TRUE)
p2 <- DimPlot(mnp, reduction = 'wnn.3.umap', group.by = 'mnp_analysis', shuffle = TRUE)
p1 + p2

# Save

In [ ]:
saveRDS(mnp, 'mnp_deep_analysis.rds')

In [ ]:
sessionInfo()